# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [126]:
# from bs4 import BeautifulSoup
# import requests

# response = requests.get("https://opensyllabus.org/results-list/titles?size=50&usState=AK")
# doc = BeautifulSoup(response.text)


from bs4 import BeautifulSoup
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; hu-HU; rv:1.7.8) Gecko/20050511 Firefox/1.0.4'}

import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [127]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/richardabbey/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/c4/ch89cvbs21d4cvs_ch64f2rh0000gn/T/ipykernel_7784/2737825784.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [128]:
#Identifying the drop down and selecting cosmetologists
Select(driver.find_element(By.ID, 'pht_status')).select_by_visible_text('Cosmetologists')

In [129]:
#Identify the textbox to input the name
driver.find_element(By.ID, 'pht_lnm').send_keys('Nguyen')

In [130]:
#Identifying the XPATH for the search button and clicking it

driver.find_element(By.XPATH,'//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, i's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [131]:
#Identifying the table holding all the rows and looping through each row

results = driver.find_elements(By.TAG_NAME, 'tr')
for result in results[:10]:
    print(result.text)

Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, DAI T
City: HOUSTON
County: Harris
Zip Code: 77034


License #: 765339

Complaint # COS20210005027 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,500. Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to store eyelash extensions in a sealed bag or covered container and kept in a clean dry debris-free s

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [132]:
#Looping through each row in the table to identify the names
#Names are held in the first element of the class name tag
#Adding extra code to identify where the code might break

for result in results[:10]:
    try:
        print(result.find_element(By.CLASS_NAME, 'results_text').text)
    except:
        print("It didn't work")

    
    

It didn't work
NGUYEN, THANH
NGUYEN, DAI T
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
NGUYEN, NAM
NGUYEN, DUC
NGUYEN, THU THAO THI


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [133]:
#Looping through the table to identify violation description
#These are held in the td tags
#Adding additional code to identify where data might not exist

for result in results[:10]:
    try:
        print("_______")
        print(result.find_elements(By.TAG_NAME, 'td')[2].text)
    except:
        print("Doesn't have a violation")

    


_______
Doesn't have a violation
_______
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
_______
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to store eyelash extensions in a sealed bag or covered container and kept in a clean dry debris-free storage area.
_______
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
__

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [134]:
#Looping through the table to identify complaint number
#These are held in the class tags
#Adding additional code to identify where data might not exist

for result in results[:10]:
    try:
        print(result.find_elements(By.CLASS_NAME, 'results_text')[5].text)
    except:
        print("Doesn't have a complaint number")

Doesn't have a complaint number
COS20210004784
COS20210005027
COS20210009745
COS20210011484
COS20210011721
COS20200007069
COS20210010530
COS20200007141
COS20200000839


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [135]:
#First step was to add an empty dictionary in the loop.
#Then I add create a variable (cells) which contains the class name in the table
#Then another variable is created to hold the tag names 
#The loop goes through and adds the respective dictionary values to the empty dictionary
#Then the dictionary is appended to the empty master list created and printed out
big_data = []

for row in results[10:]:
    all_people = {}
    cells = row.find_elements(By.CLASS_NAME, 'results_text')
    new_cells = row.find_elements(By.TAG_NAME, 'td')
    all_people ['name'] = cells[0].text
    all_people ['city'] = cells[1].text
    all_people['zip code'] = cells[3].text
    all_people ['county'] = cells[2].text
    all_people ['complaint number'] = cells[5].text
    all_people ['license number'] = cells[4].text
    all_people ['violation description'] = new_cells[2].text
    big_data.append(all_people)

big_data

[{'name': 'NGUYEN, MINH NHU',
  'city': 'DALLAS',
  'zip code': '75243',
  'county': '',
  'complaint number': 'COS20210009714',
  'license number': '766162, 1502172',
  'violation description': "Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to follow proper sequential cleaning and disinfecting procedures when using portable whirlpool jets; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to maintain powdered alum, styptic powder, or a cyanoacrylate."},
 {'name': 'NGUYEN, DUNG VAN',
  'city': 'SAULT SAINTE MARIE',
  'zip code': '497

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [136]:
#The list of dictionary created is transformed into a dataframe using df = pd.DataFrame(big_data)

big_data = []

for row in results[10:]:
    all_people = {}
    cells = row.find_elements(By.CLASS_NAME, 'results_text')
    new_cells = row.find_elements(By.TAG_NAME, 'td')
    all_people ['name'] = cells[0].text
    all_people ['city'] = cells[1].text
    all_people['zip code'] = cells[3].text
    all_people ['county'] = cells[2].text
    all_people ['complaint number'] = cells[5].text
    all_people ['license number'] = cells[4].text
    all_people ['violation description'] = new_cells[2].text
    big_data.append(all_people)

big_data

[{'name': 'NGUYEN, MINH NHU',
  'city': 'DALLAS',
  'zip code': '75243',
  'county': '',
  'complaint number': 'COS20210009714',
  'license number': '766162, 1502172',
  'violation description': "Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to follow proper sequential cleaning and disinfecting procedures when using portable whirlpool jets; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to maintain powdered alum, styptic powder, or a cyanoacrylate."},
 {'name': 'NGUYEN, DUNG VAN',
  'city': 'SAULT SAINTE MARIE',
  'zip code': '497

In [137]:
df = pd.DataFrame(big_data)

In [138]:
#df

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [139]:
#The final step was to use output to convert the dataframe into csv.
#Ignore index is used to remove the unnamed column

df.to_csv("output.csv", sep=',', index=False)

In [140]:
pd.read_csv("output.csv")

,name,city,zip code,county,complaint number,license number,violation description
0,"NGUYEN, MINH NHU",DALLAS,75243,NaN,COS20210009714,"766162, 1502172",Respondent failed to follow whirlpool foot spa...
1,"NGUYEN, DUNG VAN",SAULT SAINTE MARIE,49783,OUT OF STATE,COS20210005838,1202055,Respondent performed or attempted to perform a...
2,"NGUYEN, TINH",HOUSTON,77098,HARRIS,COS20200010129,652914,Respondents leased space in a salon to an indi...
3,"NGUYEN, HANG NU THANH",PEARLAND,77584,BRAZORIA,COS20210003441,"1642801, 803265, 1704165",Respondent failed to keep a record of the date...
4,"NGUYEN, LAN N",SPRING,77379,HARRIS,COS20200009912,726731,Respondent failed to follow whirlpool foot spa...
...,...,...,...,...,...,...,...
88,"NGUYEN, SON DANG",FOREST HILL,76119,TARRANT,COS20190010635,"720091, 1195752",Respondent failed to clean and sanitize whirlp...
89,"NGUYEN, TUYEN THANH",AUBREY,76227,DENTON,COS20190014566,"792178, 1468619, 1410422",Respondent failed to clean and sanitize whirlp...
90,"NGUYEN, STEVEN THAI",CORSICANA,75110,NAVARRO,COS20190010869,779635,Respondent failed to clean and sanitize whirlp...
91,"NGUYEN, CY VAN",HOUSTON,77007,HARRIS,COS20190007921,765590,Respondent operated a cosmetology salon withou...


In [ ]:
# Thank you